!tar -czf sourcedir.tar.gz .



!aws s3 cp sourcedir.tar.gz s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz

In [2]:
## Looking for tau parameters for backdoor trigger 
import sys, os
sys.path.append(os.path.abspath(".."))
from Detection_tools import utils as UT
import numpy as np 

print(UT)
m = 200                     # size of trigger eval set
p0 = 1 / 10                 # baseline hit rate (10 classes)
f_fp = 0.01                 # 1% false positive rate

tau_stat = UT.statistical_bound_threshold(m, p0, f_fp)
print(f"Statistical bound threshold (τ): {tau_stat:.3f}")

benign_scores = np.random.beta(2, 10, size=50)  # fake benign S scores
tau_emp = UT.empirical_quantile_threshold(benign_scores, 0.95)
print(f"Empirical quantile threshold (τ): {tau_emp:.3f}")


<module 'Detection_tools.utils' from '/home/sagemaker-user/Stinging_Nettle_Prototype/Detection_tools/utils.py'>
Statistical bound threshold (τ): 0.150
Empirical quantile threshold (τ): 0.358


# 1. Label Flipping method : 

## 1.1 Scenario I :
Single Client targeted ( client number 1) + verified clients number 0, 1, 2

### 1.1.1 20 clients:

#### 1.1.1.1 20 clients + Fedopt

In [2]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedopt-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedopt-20clients-p-2025-11-16-10-21-42-377


#### 1.1.1.2 20 clients + FedAVG

In [3]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedavg-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedavg-20clients-p-2025-11-16-17-59-49-225


### 1.1.2 50 clients:

##### 1.1.2.1 50 clients + Fedopt

In [3]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 4,
    "batch_size": 80, 
    "client_lr": 0.005,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedopt-50clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedopt-50clients-p-2025-11-17-08-05-14-521


#### 1.1.2.2 Fedavg 50 clients 

In [2]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 4,
    "batch_size": 80, 
    "client_lr": 0.005,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedavg-50clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedavg-50clients-p-2025-11-18-09-47-49-040


In [11]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 100,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 30,
    "local_epochs": 3,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.001,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.2xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedavg-100clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedavg-100clients--2025-11-07-10-53-28-362


#### 1.1.3.1 200 clients + FedOPt

In [14]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 200,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 30,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.001,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g5.xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedopt-200clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedopt-200clients--2025-11-07-10-56-46-167


#### 1.1.3.2 200 clients + FedAVG


In [15]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 200,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 30,
    "local_epochs": 3,
    "batch_size": 32, # from cifar10 64 
    "client_lr": 0.01,
    "client_momentum": 0.9,
    "weight_decay": 1e-4,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.001,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g5.xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-scenario1-fedavg-200clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-scenario1-fedavg-200clients--2025-11-07-10-56-53-259


## 1.2 Scenario II :
Subset of  clients targeted:
- in the case of 20 clients ( two clients targeted ( 0, 1) + two verified ( 1, 2))
- in the case of 50 clients (four clients targeted (0, 1, 2, 3) + four verified (2, 3, 4, 5))

### 1.2.1 20 clients

#### 1.2.1.1 Fedopt

In [3]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "label_flip",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "0 1",
    "verified_clients": "1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.p3.16xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="label-flip-sc2-fedopt-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: label-flip-sc2-fedopt-20clients-pathmni-2025-11-19-08-37-16-953


#### 1.2.1.2 FedAvg

### 1.2.2 50 clients

# 2. Backdoor

## 2.1 Scenario 1 :

### 2.1.1 20 clients:

#### 2.1.1.1 Fedopt

In [4]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedopt-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedopt-20clients-pat-2025-11-16-18-01-20-569


#### 2.1.1.2 Fedavg

In [5]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedavg-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedavg-20clients-pat-2025-11-16-18-01-51-115


### 2.1.2 50 clients

#### 2.1.2.1 Fedopt

In [3]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 4,
    "batch_size": 80, 
    "client_lr": 0.005,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedopt-50clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedopt-50clients-pat-2025-11-18-09-50-14-706


#### 2.1.2.2 Fedavg

In [4]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 4,
    "batch_size": 80, 
    "client_lr": 0.005,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "backdoor",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="backdoor-scenario1-fedavg-50clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: backdoor-scenario1-fedavg-50clients-pat-2025-11-18-09-50-48-760


# 3. Fingerprinting

### 3.1 Scenario 1 :

### 3.1.1 20 clients

#### 3.1.1.1 Fedopt

In [6]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedopt-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedopt-20clients--2025-11-16-18-04-19-340


#### 3.1.1.2 Fedavg

In [7]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 20,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 10,
    "local_epochs": 4,
    "batch_size": 64, # from cifar10 64 
    "client_lr": 0.008,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedavg-20clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedavg-20clients--2025-11-16-18-04-50-227


### 3.1.2 50 clients

#### 3.1.2.1 Fedopt

In [5]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 4,
    "batch_size": 80, 
    "client_lr": 0.005,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedopt",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedopt-50clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedopt-50clients--2025-11-18-09-52-17-933


#### 3.1.2.2 Fedavg

In [6]:

from sagemaker.huggingface import HuggingFace

role = "arn:aws:iam::711387130895:role/service-role/SageMaker-ExecutionRole-20250525T120088"

# Define hyperparameters that match your argparse arguments
hyperparameters = {
    # --- Dataset and client setup ---
    "dataset_name": "pathmnist",
    "num_clients": 50,
    "frac_clients": 1.0,
    "dirichlet_alpha": 0.5,
    "test_size": 0.2,
    "poison_size": 0.3,

    # --- Training parameters ---
    "rounds": 12,
    "local_epochs": 4,
    "batch_size": 80, 
    "client_lr": 0.005,
    "client_momentum": 0.8,
    "weight_decay": 1e-5,

    # --- Server optimisation ---
    "server_opt": "fedavg",
    "server_lr": 0.8,

    # --- Fingerprinting ---
    "enable_fingerprinting": "1",          # or False if disabled
    "fingerprint_method": "sparse",         # choices: ["sparse", "dense"]
    "fingerprint_sparsity": 0.01,
    "target_dot_strength": 1.0,

    # --- Detection / Defence params ---
    "honest_fraction": 0.1,
    "detection_margin": 1.5,
    "seed": 42,
    "history_window": 5,
    "method": "fingerprint",                 # choices: ["label_flip", "backdoor","fingerprint"]
    "label_flip_alpha": 1.0,
    "backdoor_target_label": 1,
    "backdoor_patch_size": 15,
    "backdoor_intensity": 1.0,
    "tau_backdoor_threshold_statistical": tau_stat,
    "tau_backdoor_threshold_emprical": tau_emp,


    # --- Client selection ---
    # Lists must be passed as space-separated strings for SageMaker
    "targeted_clients": "1",
    "verified_clients": "0 1 2",
}


huggingface_estimator = HuggingFace(
    entry_point="main.py",   # script inside source_dir
    source_dir="s3://poison-detect-bucket/submission2/scripts/sourcedir.tar.gz",  # zipped dir
    role=role,
    instance_type="ml.g4dn.12xlarge",  # use quota for faster training
    instance_count=1,
    transformers_version="4.36.0",
    pytorch_version="2.1.0",
    py_version="py310",
    base_job_name="fingerprint-scenario1-fedavg-50clients-pathmnist",
    hyperparameters=hyperparameters,  # pass args here
)

# Launch training
huggingface_estimator.fit(wait=False)


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: fingerprint-scenario1-fedavg-50clients--2025-11-18-09-53-36-123
